In [1]:
import pickle 
import torch 
import os 
import sys 
sys.path.append('../../../submodule') 
import torch_util 
from torch_util.imports import * 

with open(os.path.expanduser('~/dataset/HGB/processed/DBLP_hg.dict.pkl'), 'rb') as fp: 
    graph_info = pickle.load(fp)
    
graph_info 

{'node_feat_dict': {'author': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  'paper': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  'term': tensor([[-0.6924, -0.4659,  1.1540,  ...,  0.9178,  0.1995, -0.6360],
          [ 1.2031, -0.4003,  0.0740,  ...,  1.3262, -0.3325,  0.8198],
          [ 0.3748,  0.5731,  0.4802,  ...,  1.1522,  0.6010, -0.4309],
          ...,
          [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
          [ 0.1724, -0.2723, -1.3368,  ..., -0.0881,  0.0225,  0.1166],
          [ 0.2197,  0.0253,  0.1220,  ...

In [2]:
edge_index_dict = graph_info['edge_index_dict']
num_nodes_dict = graph_info['num_nodes_dict']

{ k: v.shape for k, v in edge_index_dict.items() }

{('author', 'AP', 'paper'): torch.Size([2, 19645]),
 ('paper', 'PT', 'term'): torch.Size([2, 85810]),
 ('paper', 'PV', 'venue'): torch.Size([2, 14328]),
 ('paper', 'PA', 'author'): torch.Size([2, 19645]),
 ('term', 'TP', 'paper'): torch.Size([2, 85810]),
 ('venue', 'VP', 'paper'): torch.Size([2, 14328])}

In [3]:
os.makedirs('./TransE_emb', exist_ok=True)

def convert_hg_to_triplets(edge_index_dict: dict[EdgeType, EdgeIndex], 
                           num_nodes_dict: dict[NodeType, int],
                           output_path: str): 
    nid_offset_dict: dict[NodeType, int] = dict() 

    nid_offset = 0 

    for ntype, num_nodes in sorted(num_nodes_dict.items()):
        nid_offset_dict[ntype] = nid_offset 
        nid_offset += num_nodes 

    with open(output_path, 'w', encoding='utf-8') as fp:
        for etype, edge_index in tqdm(edge_index_dict.items()): 
            src_ntype, _, dest_ntype = etype 
            etype = '__'.join(etype) 
            src_nids, dest_nids = edge_index 
            
            src_nids.add_(nid_offset_dict[src_ntype])
            dest_nids.add_(nid_offset_dict[dest_ntype])

            for src_nid, dest_nid in zip(src_nids.tolist(), dest_nids.tolist()): 
                print(f"{src_nid}\t{etype}\t{dest_nid}", file=fp)

convert_hg_to_triplets(
    edge_index_dict = edge_index_dict, 
    num_nodes_dict = num_nodes_dict,
    output_path = './TransE_emb/triplets.tsv', 
)

  0%|          | 0/6 [00:00<?, ?it/s]